In [ ]:
from discovery_utils.getters import gtr
from discovery_utils.getters import crunchbase
from discovery_utils.utils import search
from discovery_utils.utils.llm.batch_check import LLMProcessor, generate_system_message
from discovery_utils.utils import viz_landscape, analysis_crunchbase
from discovery_utils.utils.analysis_crunchbase import get_timeseries
from discovery_utils.utils import charts
from discovery_utils.utils import analysis

import asyncio

import pandas as pd

from src import PROJECT_DIR
from src import VECTOR_DB_DIR

OUTPUT_DIR = PROJECT_DIR / 'data/2025_01_MS_ahl/'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

SCORE_THRESHOLD = 0.3

CONFIGS = [
    "challenger_brands",
    "child_obesity",
    "food_as_medicine",
    "future_of_ag",
    "glp1",
    "hormonal_dysregulation",
]

In [ ]:
investment_types = ["angel", "pre_seed", "seed", "series_a", "series_b"]

In [ ]:
CB = crunchbase.CrunchbaseGetter(vector_db_path=VECTOR_DB_DIR)

In [ ]:
all_ipos_df = []
all_acquisitions_df = []
all_growth_rates = []
all_export_df = []

for config_name in CONFIGS:
    
    config_filename = f"config_MS_{config_name}.yaml"
    print(f"Processing {config_filename}")
    # Search the database with keywords and vectors
    SearchCB = search.SearchDataset(CB, CB.organisations_enriched, config_filename)
    search_cb_df = (
        SearchCB.do_search()
        # add full description text
        .merge(CB.descriptions[['id', 'description']], on='id', how='left')
        .fillna({'description': '', 'short_description': '', 'name': ''})
        .assign(text = lambda df: df['name'] + '. ' + df['short_description'] + ' ' + df['description'])
    )    

    # Filter the results to only include those with a score above a threshold
    relevant_df = search_cb_df.query(f"_score_avg > {SCORE_THRESHOLD}")

    system_message = generate_system_message(config_filename)
    fields = [
        {"name": "is_relevant", "type": "str", "description": "A one-word answer: 'yes' or 'no'."},
    ]
    check_data = dict(zip(relevant_df['id'], relevant_df['text']))

    processor = LLMProcessor(
        output_path=str(OUTPUT_DIR / f"llm_check_MS_{config_name}.jsonl"),
        system_message=system_message,
        session_name="mission_studio",
        output_fields=fields,
    )

    await processor.run(check_data, batch_size=15, sleep_time=0.5)   
    
    relevant_check_df = pd.read_json(OUTPUT_DIR / f"llm_check_MS_{config_name}.jsonl", lines=True)
    relevant_checked_df = relevant_df.merge(relevant_check_df[['id', 'is_relevant']], left_on='id', right_on='id', how='left')
    matching_ids = relevant_checked_df.query("is_relevant == 'yes'").id.tolist()

    # IPOs and acquisitions
    ipos_df = CB.ipos.query("org_id in @matching_ids")
    acquisitions_df = CB.acquisitions.query("acquiree_id in @matching_ids")

    if len(ipos_df) > 0:
        ipos_df.to_csv(OUTPUT_DIR / f"ipos_{config_name}.csv", index=False)
        all_ipos_df.append(ipos_df.assign(theme=config_name))

    if len(acquisitions_df) > 0:
        acquisitions_df.to_csv(OUTPUT_DIR / f"acquisitions_{config_name}.csv", index=False)
        all_acquisitions_df.append(acquisitions_df.assign(theme=config_name)) 

    # Time series analysis
    ts_df = get_timeseries(
        cb_orgs = CB.organisations_enriched.query("id in @matching_ids"),
        cb_funding_rounds = CB.funding_rounds_enriched.query("org_id in @matching_ids").drop_duplicates("funding_round_id").query("investment_type in @investment_types"),
        min_year = 2010,
        max_year = 2025,
        period='year',
    )
    growth_rates = analysis.smoothed_growth(ts_df, year_start=2020, year_end=2024)
    growth_rates_df = pd.DataFrame(growth_rates, columns=[config_name]).T.reset_index().rename(columns={'index': 'theme'})
    all_growth_rates.append(growth_rates_df)

    if len(matching_ids) > 20:            
        # Generate the landscapes
        id_condition = "id in ('{}')".format("', '".join(list(matching_ids)))
        vectors_df = CB.VectorDB.vector_db.search().where(id_condition).limit(30000).to_pandas()    

        fig, cb_viz_df = viz_landscape.generate_crunchbase_landscape(vectors_df, CB, min_cluster_size=15, n_keyword_clusters=10)

        output_path = f"cb_landscape_{config_name}.html"
        fig.save(str(output_path))    
        #cb_viz_df.to_csv(f"cb_landscape_{config_name}.csv", index=False)

        export_df = (
            relevant_checked_df
            .merge(cb_viz_df[['id', 'category', 'keyword_cluster', 'recent_funding', 'region']], on='id', how='left')
        )   
        export_df.to_csv(OUTPUT_DIR / f"cb_landscape_data_{config_name}.csv", index=False)
        all_export_df.append(export_df.assign(theme=config_name))
    else:
        print(f"Not enough companies to generate landscape for {config_name}")
        relevant_checked_df.to_csv(OUTPUT_DIR / f"cb_landscape_data_{config_name}.csv", index=False)


all_ipos_df = pd.concat(all_ipos_df, ignore_index=True).to_csv(OUTPUT_DIR / "all_ipos.csv", index=False)
all_acquisitions_df = pd.concat(all_acquisitions_df, ignore_index=True).to_csv(OUTPUT_DIR / "all_acquisitions.csv", index=False)

all_growth_rates_df = pd.concat(all_growth_rates, ignore_index=True)
all_growth_rates_df.to_csv(OUTPUT_DIR / "growth_rates.csv", index=False)

all_export_df = pd.concat(all_export_df, ignore_index=True)
all_export_df.to_csv(OUTPUT_DIR / "all_cb_landscape_data.csv", index=False)

## Baseline calculations

In [ ]:
baseline_df = get_timeseries(
    cb_orgs = CB.organisations_enriched,
    cb_funding_rounds = CB.funding_rounds_enriched.drop_duplicates("funding_round_id").query("investment_type in @investment_types"),
    min_year = 2010,
    max_year = 2025,
    period='year',
)

In [ ]:
baseline_df

In [ ]:
baseline_growth_rates = analysis.smoothed_growth(baseline_df, year_start=2020, year_end=2024)
baseline_growth_rates = pd.DataFrame(baseline_growth_rates, columns=["baseline"]).T.reset_index().rename(columns={'index': 'baseline'})
baseline_growth_rates.to_csv(OUTPUT_DIR / "baseline_growth_rates.csv", index=False)

In [ ]:
baseline_growth_rates

In [ ]:
fig = charts.ts_bar(
    ts=baseline_df,
    variable = "raised_amount_gbp_total",
    variable_title = "Total global investment"
)
# save png with altair
fig.save(str(OUTPUT_DIR / "v2_baseline_investment.png"), scale_factor=2.0)

In [ ]:
fig = charts.ts_bar(
    ts=baseline_df,
    variable = "n_orgs_founded",
    variable_title = "Number of organisations founded"
)
fig.save(str(OUTPUT_DIR / "v2_baseline_new_companies.png"), scale_factor=2.0)

In [ ]:
funding_rounds_df = CB.funding_rounds_enriched.drop_duplicates("funding_round_id").query("investment_type in @investment_types")

In [ ]:
deals_df, deal_counts_df = analysis_crunchbase.get_funding_by_year_and_range(funding_rounds_df, 2014, 2025)
aggregated_funding_types_df = analysis_crunchbase.aggregate_by_funding_round_types(funding_rounds_df.query("year >= 2014"))

In [ ]:
investment_types_fig = analysis_crunchbase.chart_investment_types(aggregated_funding_types_df)
investment_types_fig.save(str(OUTPUT_DIR / "v2_baseline_investment_types.png"), scale_factor=2.0)

In [ ]:
investment_types_counts_fig = analysis_crunchbase.chart_investment_types_counts(aggregated_funding_types_df)
investment_types_counts_fig.save(str(OUTPUT_DIR / "v2_baseline_investment_types_counts.png"), scale_factor=2.0)

In [ ]:
deal_sizes_counts_fig = analysis_crunchbase.chart_deal_sizes_counts(deal_counts_df)
deal_sizes_counts_fig.save(str(OUTPUT_DIR / "v2_baseline_deal_sizes_counts.png"), scale_factor=2.0)

In [ ]:
deal_sizes_fig = analysis_crunchbase.chart_deal_sizes(deals_df)
deal_sizes_fig.save(str(OUTPUT_DIR / "v2_baseline_deal_sizes.png"), scale_factor=2.0)